In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import color
from time import time
from scipy import signal
from tqdm.autonotebook import tqdm
import copy
import imageio

In [ ]:
img = plt.imread('/content/drive/MyDrive/COL783/1024px-Broadway_tower_edit.jpg')

In [ ]:
plt.imshow(img)
plt.title('Original Image')

In [ ]:
print(img.shape)

In [ ]:
def energyfunction(img): 
  img1 = np.copy(img)
  img1 = color.rgb2gray(img1)
  #finding the gradient in x and y dir
  sobelx = cv2.Sobel(img1,cv2.CV_64F,1,0,ksize=5)
  sobely = cv2.Sobel(img1,cv2.CV_64F,0,1,ksize=5)
  #energy function
  energy = abs(sobelx) + abs(sobely)
  return energy

def computeSeam(energy):
  dp = np.copy(energy)
  paths = np.zeros(dp.shape)
  for i in range(1,dp.shape[0]):
    for j in range(dp.shape[1]):
      if (j>=1 and j+1<dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]]))
      elif (j+1>=dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j]]))
      elif (j<1):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(1 + np.argmin([dp[i-1,j],dp[i-1,j+1]]))
        
  return dp,paths

def plotseam(img,paths,col):
  img3 = np.copy(img)
  for i in range(img.shape[0]):
    indx = np.int(img.shape[0]-1-i)
    img3[indx,col] = [255,0,0]
    col = np.int(col + paths[indx,col] - 1)
    
  plt.figure(figsize=(10,10))
  plt.title('Seam')
  plt.imshow(img3)
  plt.show()
  return img3

def removeSeam(img):
  #first compute the energy function
  energy = energyfunction(img)
  #visualize the energy function
  plt.figure(figsize=(10,10))
  plt.imshow(energy,cmap='gray')
  plt.title('Energy Function')
  plt.show()
  #dynamic programming algorithm to calculate cum energy
  dp,paths = computeSeam(energy)
  #find min seam
  col = np.int(np.argmin(dp[dp.shape[0]-1,:]))
  #visulaizing the seam
  img3 = plotseam(img,paths,col)
  #img2 will store the image after the removal of the seam
  img2 = np.zeros((img.shape[0],img.shape[1] - 1,3))

  for i in range(img2.shape[0]):
    indx = np.int(img2.shape[0]-1-i)
    if (col+1<img.shape[1]):
      img2[indx][0:col] = img[indx][0:col]
      img2[indx][col:] = img[indx][1+col:]
    else:
      img2[indx] = img[indx,:col]
    col = np.int(col + paths[indx,col] - 1)

  print("Image size after seam removal: ",img2.shape[0],img2.shape[1])
  return img2.astype('uint8'),img3.astype('uint8')

In [ ]:
#make image dimensions n1 and m1
def retarget(n1,m1,img):
  image_list = []
  print(type(img))
  print('The size of the original image: ',img.shape[0],img.shape[1])
  n = img.shape[0]  
  m = img.shape[1]
  c = n - n1
  r = m - m1
  print('Number of vertical seam removal: ',c)
  print('Number of horizontal seam removal: ',r)
  img1 = np.copy(img)
  #remove the vertical seams
  print('Removing vertical seams')
  for i in tqdm(range(r)):
    print('Iteration: ',i)
    img1,img3 = removeSeam(img1)
    image_list.append(img1)
    image_list.append(img3)

  img2 = np.copy(img1)
  img2 = np.rot90(img2)
  print('Removing Horizontal seams')
  for j in tqdm(range(c)):
    img2,img3 = removeSeam(img2)
    image_list.append(img2)
    image_list.append(img3)

  img2 = np.rot90(img2)
  img2 = np.rot90(img2)
  img2 = np.rot90(img2)

  print('The size of the image is: ',img2.shape[0],img2.shape[1])
  plt.figure(figsize=(10,10))
  plt.imshow(img2)
  plt.show()
  return img2,image_list


In [ ]:
img1 = np.copy(img)
start = time()
img2,image_list = retarget(674,1004,img1)
end = time()
print('The time elapsed in normal method: ',(end - start)/60,' minutes')

In [ ]:
plt.imshow(img2)
plt.title('Result')
print('Size of the image: ',img2.shape)

In [ ]:
#number of vertical seams: dy *2: 0 to 47
#number of horizontal seams: dx*2: 48 - 57
dx = img.shape[0] - 674
dy = img.shape[1] - 1004
fname = []
for i in range(2*dy):
  print('Image: ',i)
  fname.append(image_list[i])
  plt.imshow(image_list[i])
  plt.show()

for i in range(2*dx):
  print('Image: ',i + 2*dy)
  im = image_list[i + 2*dy]
  im = np.rot90(im)
  im = np.rot90(im)
  im = np.rot90(im)
  fname.append(im)
  plt.imshow(im)
  plt.show()

In [ ]:
imageio.mimsave('seam_retarget.gif',fname)

In [ ]:
#number of vertical seams: dy *2: 0 to 47
#number of horizontal seams: dx*2: 48 - 57
print(dx,dy)

In [ ]:
#Fast seam method
start = time()
!python3 "/content/drive/MyDrive/COL783/main_reduce.py" 20 20 
end = time()
print('Fast seam time elapsed: ',(end - start),' seconds')

In [ ]:
result = plt.imread('/content/drive/MyDrive/COL783/fastseamresult.jpg')
plt.figure(figsize=(10,10))
plt.imshow(result)
print('The size of image: ',result.shape)